In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0039310455322265625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 22.818970177507452
200000 41.640952347835196
300000 48.29182319808874
400000 48.55367416681641
500000 49.574338620416
600000 48.79269012573799
700000 47.3728031716832
800000 47.59444929981052
900000 47.05261270044611
1000000 16.30518689234797
1100000 48.43893373386063
1200000 46.90684268105812
1300000 47.03547229049012
1400000 46.57731092834874
1500000 49.14033355265662
1600000 46.79593659542231
1700000 47.13045474362759
1800000 47.45358518120569
1900000 40.25672056148877
2000000 23.32831685116732
2100000 45.586205578722144
2200000 45.21406524335722
2300000 44.29002803981077
2400000 45.266917385044
2500000 45.07331263737886
2600000 43.13157894735162
2700000 43.7766267355448
2800000 43.5152828205651
2900000 18.285153565003935
3000000 42.85882950926529
310

29800000 40.76834129345381
29900000 41.18519525790135
30000000 43.80336402392686
30100000 43.508884509265926
30200000 43.78243300225203
30300000 27.05172063667798
30400000 38.8603178890847
30500000 44.74256815178132
30600000 46.028470897341805
30700000 45.85701953716435
30800000 20.493662230803523
30900000 46.793078025373546
31000000 47.07224806952172
31100000 45.1856030073768
31200000 45.43561027813351
31300000 42.07808748406251
31400000 41.68740365089296
31500000 16.990626452699313
31600000 41.55841625178777
31700000 41.01404890060884
31800000 43.47337038143161
31900000 44.07959885718078
32000000 46.29290336073396
32100000 46.47169303054216
32200000 24.84573454513212
32300000 43.07668531181691
32400000 45.9707495599834
32500000 45.62409063669045
32600000 44.65302012678612
32700000 27.21628653195434
32800000 38.360990571207836
32900000 46.13839536018537
33000000 42.62237935244876
33100000 42.46339700586089
33200000 40.28999946506866
33300000 38.576944873846344
33400000 19.569961220177

59900000 36.2645160778209
60000000 37.12547621096128
60100000 14.994218123005187
60200000 37.94715615831841
60300000 36.09939276855445
60400000 37.48159124660874
60500000 36.82848822575227
60600000 20.171947779630553
60700000 39.17153992777636
60800000 39.976387933143634
60900000 40.04338104894847
61000000 43.80061961877028
61100000 43.208060289882944
61200000 42.07570270862017
61300000 42.09694275305462
61400000 41.35083034847379
61500000 33.53243974807352
61600000 24.932598955735653
61700000 37.55612082913783
61800000 40.08142193330688
61900000 37.33229691818246
62000000 17.75581760414755
62100000 35.27178507201819
62200000 39.751109094342965
62300000 36.45175188986231
62400000 38.59346239323944
62500000 21.24554440487503
62600000 40.28645468338503
62700000 41.722719812790814
62800000 42.40421455527576
62900000 42.28723609406679
63000000 41.50942603287027
63100000 40.72163857019825
63200000 40.59156784626336
63300000 38.93832324739725
63400000 38.9966183499222
63500000 17.57245593682

90000000 36.36656647612237
90100000 37.09478058563498
90200000 36.70530266752481
90300000 35.94339606009559
90400000 17.030379613456326
90500000 33.458770526792904
90600000 16.813576490635814
90700000 32.974991788960004
90800000 27.864889088706374
90900000 26.720993856276603
91000000 36.52390786237186
91100000 35.61972239657596
91200000 36.4939179876047
91300000 36.374808831066446
91400000 39.278215190971096
91500000 39.136653079434424
91600000 40.87176450428372
91700000 40.24861837963277
91800000 38.45400202899545
91900000 36.5587644911038
92000000 37.54543658481742
92100000 36.77268571653994
92200000 36.95241863187628
92300000 16.709581152365892
92400000 36.79913476653409
92500000 17.84217232219835
92600000 32.26378470694226
92700000 25.206061989198886
92800000 29.86217567218958
92900000 36.23611789618628
93000000 36.68503259848262
93100000 36.54360780933981
93200000 37.03470926278257
93300000 39.42108557772494
93400000 40.68782726092272
93500000 40.54138855991352
93600000 39.0463405

119300000 3.9034384253322583
119400000 3.914214405365676
119500000 3.876881053160777
119600000 3.8615310698820418
119700000 3.868787081734935
119800000 3.8407238315045835
119900000 3.8940457029024396
120000000 3.8966089092942076
120100000 3.887492873732236
120200000 3.8588364709912932
120300000 3.863732586643249
120400000 4.265907365108464
120500000 3.9270380014289916
120600000 3.9429386621293623
120700000 3.91625078394309
120800000 3.8966070026255255
120900000 3.904608707536807
121000000 3.8935410100744963
121100000 3.881864966498163
121200000 3.8914273491048212
121300000 3.8891416221185495
121400000 3.868515598680218
121500000 3.9008359924855927
121600000 3.9015114289704065
121700000 3.8757481498813
121800000 3.8809063979195257
121900000 3.8924383224567474
122000000 3.8906245775594077
122100000 3.884257236484915
122200000 3.881914103891068
122300000 3.882760045100377
122400000 3.854218962687135
122500000 4.257254889748113
122600000 4.369646784635183
122700000 3.9184085989978015
12280

...

493200000 3.7775959573437974
493300000 3.5483753513709044
493400000 1.4527505228955413
493500000 3.4143889233673113
493600000 2.9978097085684703
493700000 2.8176573855807616
493800000 3.402096250016034
493900000 3.6853168592638506
494000000 3.675516309854415
494100000 4.032961974725595
494200000 3.508613205427031
494300000 2.3400966017454268
494400000 2.497233538832476
494500000 3.1886650810469024
494600000 2.4016210372378497
494700000 3.429970436680406
494800000 3.7100197654306357
494900000 3.765603595258554
495000000 3.5649911097621425
495100000 3.856060239170846
495200000 3.61583184165849
495300000 1.262287389969309
495400000 3.27844856312913
495500000 2.225409133127527
495600000 3.4188175977867843
495700000 3.49672563981188
495800000 3.5845645488228732
495900000 3.4352616649517813
496000000 3.4312879451859164
496100000 3.7686023116946585
496200000 2.377246101123204
496300000 2.3632478016137646
496400000 2.579755308668554
496500000 3.1267276654648484
496600000 3.283823789855202
4967

521200000 2.869236004027446
521300000 2.8439980781243475
521400000 2.9348951991898513
521500000 3.0818736218258307
521600000 3.016400241034223
521700000 2.9438375576161144
521800000 3.037205621220407
521900000 3.052040295867059
522000000 3.173653863929729
522100000 3.0872549436605965
522200000 3.0548218794850404
522300000 3.1055207456772127
522400000 3.177942210482184
522500000 3.157696571803861
522600000 3.1174015096465353
522700000 3.066985628389544
522800000 3.180590532253865
522900000 3.114269037695815
523000000 3.1008767727397077
523100000 2.989574524980078
523200000 3.170021998887002
523300000 3.0654055056678056
523400000 3.093062896102508
523500000 2.7689272304413315
523600000 2.706837734557821
523700000 2.7094213840121526
523800000 2.8979167783324717
523900000 2.940156567788356
524000000 2.9652790668515827
524100000 2.8053249309108255
524200000 2.996565255882512
524300000 2.920210049145066
524400000 3.6705289774241145
524500000 3.4159753148310945
524600000 3.5739757147438556
52

548000000 1.7698879499973457
548100000 2.76334740143163
548200000 3.1137670948083134
548300000 1.0581066755338344
548400000 0.5692070973379214
548500000 0.5653127853363256
548600000 0.5726852823240491
548700000 0.5664086540882289
548800000 0.5606848095792433
548900000 0.5648443563420374
549000000 0.5701378280752003
549100000 0.5707457728008065
549200000 0.5717558342608247
549300000 0.5618553828979234
549400000 0.5590313106865817
549500000 0.5565234203166465
549600000 0.5544079053474963
549700000 0.5533263321146336
549800000 0.5531739657722943
549900000 0.5520517496088346
550000000 0.5513369924585522
550100000 0.5509347746175196
550200000 0.5537169850648443
550300000 0.5606189965045021
550400000 0.5546737694611019
550500000 0.5539997163171172
550600000 0.5737072414364451
550700000 3.460250373743464
550800000 3.305482485363593
550900000 1.4141833733377391
551000000 3.3666418211129074
551100000 3.4153554206627312
551200000 2.8063206370338594
551300000 2.358174307660414
551400000 1.5368281

576300000 1.3210792873927655
576400000 1.2705083003908295
576500000 1.0660449929021332
576600000 0.7508851192172766
576700000 1.1788358148854376
576800000 1.3435417392310633
576900000 1.0643873703594309
577000000 1.2379869550329017
577100000 0.6199933635211952
577200000 1.3674797181651692
577300000 1.3348590495233732
577400000 0.9909220272906623
577500000 1.1836562065275014
577600000 0.7014051022164159
577700000 1.3335566537150376
577800000 1.1823111501051387
577900000 1.0877945767250392
578000000 0.9831312978298664
578100000 0.8940901774626175
578200000 1.3532857950557307
578300000 0.9683720907105903
578400000 1.2867574278363882
578500000 0.7839955372799238
578600000 1.1521896760868298
578700000 1.360534547538936
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.43687323564766506
578900000 0.29340518561503953
579000000 0.29251740518989167
579100000 0.2918632600700955
579200000 0.2888330242203673
579300000 0.2857624509061873
579400000 0.2851036629615294
5

599700000 0.1694564145499766
599800000 0.43069178098989525
599900000 0.79953123738472
600000000 0.9126232497332394
600100000 0.9225345913887719
600200000 0.6072648166847195
600300000 0.8637828359793523
600400000 0.5387026295100947
600500000 0.8429386952851448
600600000 0.7738935337426861
600700000 0.6448705957546003
600800000 0.8453958349238269
600900000 0.5503985160134137
601000000 0.8390738652228613
601100000 0.6389425887638462
601200000 0.7372588093951542
601300000 0.7366214357060791
601400000 0.5992726857929329
601500000 0.82552423540017
601600000 0.5764774034484526
601700000 0.7790354719994432
601800000 0.6601717923270464
601900000 0.652484370395667
602000000 0.7590866207926207
602100000 0.6013658933587671
602200000 0.835765924672521
602300000 0.5837574494739605
602400000 0.7191407689769963
602500000 0.7293875012096663
602600000 0.5500309014127056
602700000 0.7359120298190713
602800000 0.5615163879295256
602900000 0.6715697232072949
603000000 0.570384293015778
603100000 0.66116241